In [1]:
import os
import math
import pandas as pd
import numpy as np
from scipy.signal import butter, lfilter, find_peaks, tukey
from scipy.stats import mode
from ssqueezepy import ssq_cwt
import matplotlib.pyplot as plt

In [2]:
import time

In [3]:
def bin_data(data,fs):
  t_seq = np.array(data['timestamp'])/1000
  t0 = t_seq[0]//3600*3600
  data['num'] = np.floor((t_seq - t0)*fs)
  data = data.drop(['timestamp'], axis=1)
  m_xyz = data.groupby('num').mean()
  mag = np.array(np.sqrt(m_xyz['x']**2+m_xyz['y']**2+m_xyz['z']**2))
  if np.mean(mag)>6:
    mag = mag/9.8
  stamp = np.unique(data['num'])/fs
  return stamp,mag

In [4]:
def split_burst(stamp):
  burst_index = []
  t_diff   = (stamp[1:]-stamp[:-1])>1
  t_breaks = [j for j,val in enumerate(t_diff) if val]
  t_breaks.insert(0,0)
  t_breaks.append(len(stamp))
  for i in range(len(t_breaks)-1):
    burst_index.append([t_breaks[i],t_breaks[i+1]])
  return burst_index

## bandpass filter
def bandpass_filter(burst, lowcut, highcut, order, fs):
  b, a = butter(order, [lowcut, highcut], btype='bandpass',fs=fs)
  y = lfilter(b, a, burst)
  return y

def adjust_burst(tapered_burst,fs,edge):
  burst = tapered_burst[edge*fs:-edge*fs]
  if len(burst)%fs>=0.7*fs:
    for i in range(fs-len(burst)%fs):
      burst = np.append(burst,burst[-1])
  elif len(burst)%fs!=0:
    burst = burst[np.arange(len(burst)//fs*fs)]
  return burst

def adjust_out0(out,fs,edge):
  temp = out[0].copy()
  temp = temp[:,edge*fs:-edge*fs]
  if temp.shape[1]%fs>=0.7*fs:
    for i in range(fs-temp.shape[1]%fs):
      temp = np.append(temp,out[0][:,-1].reshape(-1,1),1)
  elif temp.shape[1]%fs!=0:
    temp = temp[:,np.arange(temp.shape[1]//fs*fs)]
  return np.array(temp)

def get_walk_prop(all_WI):
  total_len = 0
  total_walk = 0
  for i in range(len(all_WI)):
    WI = all_WI[i]
    total_len = total_len + len(WI)
    total_walk = total_walk + sum(WI)
  return total_walk/(total_len+0.001)

In [5]:
## function checks if identified peaks appear consistently in time (T)
## and frequency (delta) domains
def findContinuousDominantPeaks(E,T,delta):
  B = np.zeros((E.shape[0],E.shape[1]))
  for m in range(E.shape[1]-T):
    A = E[:,np.arange(m,m+T)]
    loop = [i for i in np.arange(T)] + [i for i in np.arange(T-2,-1,-1)]
    for t in range(len(loop)):
      s = loop[t]
      pr = np.where(A[:,s]!=0)[0]
      j = 0
      if len(pr)>0:
        for i in range(len(pr)):
          index = np.arange(max(0,pr[i]-delta),min(pr[i]+delta+1,A.shape[0]))
          if s == 0 or s == T-1:
            W = np.transpose(np.array([np.ones(len(index))*pr[i],index],dtype=int))
          else:
            W = np.transpose(np.array([index,np.ones(len(index))*pr[i],index],dtype=int))
          F = np.zeros((W.shape[0],W.shape[1]),dtype=int)
          if s == 0:
            F[:,0] = A[W[:,0],s]
            F[:,1] = A[W[:,1],s+1]
          elif s == T-1:
            F[:,0] = A[W[:,0],s]
            F[:,1] = A[W[:,1],s-1]
          else:
            F[:,0] = A[W[:,0],s-1]
            F[:,1] = A[W[:,1],s]
            F[:,2] = A[W[:,2],s+1]
          G1 = W[np.sum(F[:,np.arange(2)],axis=1)>1,:]
          if s == 0 or s == T-1:
            if G1.shape[0]==0:
              A[W[:,0],s] = 0
            else:
              j = j + 1
          else:
            G2 = W[np.sum(F[:,np.arange(1,3)],axis=1)>1,:]
            if G1.shape[0]==0 or G2.shape[0]==0:
              A[W[:,1],s] = 0
            else:
              j = j + 1
      if j == 0:
        A = np.zeros((A.shape[0],A.shape[1]))
        break
    B[:,np.arange(m,m+T)] = np.maximum(B[:,np.arange(m,m+T)],A)
  return B

In [6]:
def morse_step_count_burst(burst,fs,lowcut,highcut,order,min_amp,step_freq,alpha,beta,T,delta,wave,edge):
  w = tukey(len(burst), alpha=0.02, sym=True) 
  tapered_burst = np.concatenate((np.zeros(edge*fs),(burst-1)*w,np.zeros(edge*fs)))
  wavelet = ('gmw', {'beta': wave,'gamma':3})
  try:
    out = ssq_cwt(tapered_burst,wavelet,fs=10)
  except:
    tapered_burst = tapered_burst[:-1]
    out = ssq_cwt(tapered_burst,wavelet,fs=10)
  temp = adjust_burst(tapered_burst,fs,edge)
  dur = len(temp)/fs
  filtered_burst = bandpass_filter(temp, lowcut, highcut, order, fs)
  x = filtered_burst.reshape(fs,-1,order="F") 
  pp = np.array([max(x[:,i])-min(x[:,i]) for i in range(x.shape[1])])
  valid = np.ones(len(pp),dtype=bool)
  valid[pp<min_amp]=False 
  index = (out[2]>0.6)*(out[2]<4.8)
  freqs = out[2][index]
  coefs = adjust_out0(out,fs,edge)[index,:]
  coefs2 = np.abs(coefs**2)
  D = np.zeros((coefs2.shape[0],int(coefs2.shape[1]/fs)))
  loc_min = np.argmin(abs(freqs-step_freq[0]))
  loc_max = np.argmin(abs(freqs-step_freq[1]))
  for i in range(int(coefs2.shape[1]/fs)):
    # segment measurement into one-second non-overlapping windows
    xStart  = i*fs
    xFinish = (i+1)*fs
    # identify peaks and their l ocation in each window
    window = np.sum(coefs2[:,np.arange(xStart,xFinish)],axis=1)
    locs,_ = find_peaks(window)
    pks = window[locs]
    I = np.argsort(-pks)
    locs = locs[I]
    pks = pks[I]
    index_in_range = []
    for j in range(len(locs)):
      if locs[j]>=loc_min and locs[j]<=loc_max:
        index_in_range.append(j)
      if len(index_in_range)>=1:
        break
    x = np.zeros(coefs2.shape[0])
    if len(index_in_range)>0:
      if locs[0]>loc_max:
        if pks[0]/pks[index_in_range[0]]<alpha:
          x[locs[index_in_range[0]]]=1
      elif locs[0]<loc_min:
        if pks[0]/pks[index_in_range[0]]<beta:
          x[locs[index_in_range[0]]]=1
      else:
        x[locs[index_in_range[0]]]=1
    D[:,i]=x
  E = np.zeros((D.shape[0],D.shape[1]))
  E[:,valid] = D[:,valid]
  B = findContinuousDominantPeaks(E,T,delta)
  WI = np.zeros(E.shape[1])
  e = np.sum(B,axis=0)
  WI[e>0]=1
  cad = np.zeros(E.shape[1])
  for i in range(len(cad)):
    ind_freqs = np.where(B[:,i]>0)[0]
    if len(ind_freqs)>0:
      cad[i] = freqs[ind_freqs[0]]
  return int(round(sum(cad),0)),int(dur),WI

In [7]:
def step_count_hour(data,fs,err,lowcut,highcut,order,min_amp,step_freq,alpha,beta,T,delta,wave,edge):
  stamp,mag = bin_data(data,fs)
  burst_index = split_burst(stamp)
  step_hour = []
  dur_hour = []
  minute_hour = []
  on_person_hour = []
  if len(burst_index)>0:
    for i in range(len(burst_index)):
      if burst_index[i][1] - burst_index[i][0] > 40:
        burst = mag[np.arange(burst_index[i][0],burst_index[i][1])]
        interval = stamp[np.arange(burst_index[i][0],burst_index[i][1])]
        minute = mode(interval//60)[0][0]
        on_person = 1 if np.std(burst)> err else 0
        step,dur,WI = morse_step_count_burst(burst,fs,lowcut,highcut,order,min_amp,step_freq,alpha,beta,T,delta,wave,edge)
        step_hour.append(step)
        dur_hour.append(dur)
        minute_hour.append(minute)
        on_person_hour.append(on_person)
  return step_hour,dur_hour,minute_hour,on_person_hour

In [8]:
fs = 10
lowcut = 0.6
highcut = 4.6
order = 3
min_amp = 0.2
step_freq = [1.2,2.3]
alpha = 4
beta = 2
T = 5
delta = 3
wave = 70
edge = 5
err = 0.005

In [9]:
## test on a dataset
data = pd.read_csv("C:/Users/glius/Google Drive/Thesis/paper 3/walk_dataset/1/acc_walking_thigh.csv")
data.columns = ["id","timestamp","x","y","z"]
data.head(5)

,id,timestamp,x,y,z
0,1,1435993159419,0.055665,9.621099,-0.011372
1,2,1435993159437,0.073622,9.554062,-0.077812
2,3,1435993159456,0.077812,9.537901,-0.155025
3,4,1435993159475,0.113725,9.567230,-0.159214
4,5,1435993159499,0.116717,9.599552,-0.104746


In [10]:
t0 = time.time()
stamp,mag = bin_data(data,fs)
t1 = time.time()
stamp, t1-t0

(array([3559.4, 3559.5, 3559.6, ..., 4198.7, 4198.8, 4198.9]),
 0.03340911865234375)

In [11]:
t0 = time.time()
burst_index = split_burst(stamp)
t1 = time.time()
burst_index, t1-t0

([[0, 908],
  [908, 962],
  [962, 1017],
  [1017, 2893],
  [2893, 2985],
  [2985, 3119],
  [3119, 3197],
  [3197, 4877],
  [4877, 5010],
  [5010, 5226],
  [5226, 5344],
  [5344, 6214]],
 0.0)

In [12]:
i = 5
burst = mag[np.arange(burst_index[i][0],burst_index[i][1])]
morse_step_count_burst(burst,fs,lowcut,highcut,order,min_amp,step_freq,alpha,beta,T,delta,wave,edge)

(19, 13, array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0.]))

In [16]:
## test on upstairs
## test on two individuals and locations
final_steps = np.zeros((10,7))
final_props = np.zeros((10,7))
for i in range(2):
  path0 = "C:/Users/glius/Downloads/har/subject"+str(i+1)+"/"
  for k in range(len(os.listdir(path0))):
    path = path0 + os.listdir(path0)[k] +"/"
    for file in os.listdir(path):
      if "acc" in file:
        print(i,k,file)
        data = pd.read_csv(path+file)
        data.columns = ["id","timestamp","x","y","z"]
        stamp,mag = bin_data(data,fs)
        burst_index = split_burst(stamp)
        total_step = 0
        all_WI = []
        for j in range(len(burst_index)):
          burst = mag[np.arange(burst_index[j][0],burst_index[j][1])]
          step,dur,WI = morse_step_count_burst(burst,fs,lowcut,highcut,order,min_amp,step_freq,alpha,beta,T,delta,wave,edge)
          total_step = total_step + step
          all_WI.append(WI)
        total_prop = get_walk_prop(all_WI)
        if "chest" in file:
          final_steps[5*i+k,0] = total_step
          final_props[5*i+k,0] = total_prop
        elif "forearm" in file:
          final_steps[5*i+k,1] = total_step
          final_props[5*i+k,1] = total_prop
        elif "head" in file:
          final_steps[5*i+k,2] = total_step
          final_props[5*i+k,2] = total_prop
        elif "shin" in file:
          final_steps[5*i+k,3] = total_step
          final_props[5*i+k,3] = total_prop
        elif "thigh" in file:
          final_steps[5*i+k,4] = total_step
          final_props[5*i+k,4] = total_prop
        elif "upperarm" in file:
          final_steps[5*i+k,5] = total_step
          final_props[5*i+k,5] = total_prop
        else:    ## waist
          final_steps[5*i+k,6] = total_step
        final_props[5*i+k,6] = total_prop
np.savetxt("C:/Users/glius/Downloads/walk_step.csv", final_steps, delimiter=",")
np.savetxt("C:/Users/glius/Downloads/walk_prop.csv", final_props, delimiter=",")

0 0 acc_climbingdown_chest.csv
0 0 acc_climbingdown_forearm.csv
0 0 acc_climbingdown_head.csv
0 0 acc_climbingdown_shin.csv
0 0 acc_climbingdown_thigh.csv
0 0 acc_climbingdown_upperarm.csv
0 0 acc_climbingdown_waist.csv
0 1 acc_jumping_chest.csv
0 1 acc_jumping_forearm.csv
0 1 acc_jumping_head.csv
0 1 acc_jumping_shin.csv
0 1 acc_jumping_thigh.csv
0 1 acc_jumping_upperarm.csv
0 1 acc_jumping_waist.csv
0 2 acc_running_chest.csv
0 2 acc_running_forearm.csv
0 2 acc_running_head.csv
0 2 acc_running_shin.csv
0 2 acc_running_thigh.csv
0 2 acc_running_upperarm.csv
0 2 acc_running_waist.csv
0 3 acc_climbingup_chest.csv
0 3 acc_climbingup_forearm.csv
0 3 acc_climbingup_head.csv
0 3 acc_climbingup_shin.csv
0 3 acc_climbingup_thigh.csv
0 3 acc_climbingup_upperarm.csv
0 3 acc_climbingup_waist.csv
0 4 acc_walking_chest.csv
0 4 acc_walking_forearm.csv
0 4 acc_walking_head.csv
0 4 acc_walking_shin.csv
0 4 acc_walking_thigh.csv
0 4 acc_walking_upperarm.csv
0 4 acc_walking_waist.csv
1 0 acc_climbingdow